<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_E1_Cassandra_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install Cassandra

install https://phoenixnap.com/kb/install-cassandra-on-ubuntu

In [6]:
!apt update > null
!apt install openjdk-8-jdk -y > null
#!java -version
!apt install apt-transport-https > null

In [7]:
!sh -c 'echo "deb http://www.apache.org/dist/cassandra/debian 40x main" > /etc/apt/sources.list.d/cassandra.list'
!wget -q -O - https://www.apache.org/dist/cassandra/KEYS | sudo apt-key add -

OK


In [ ]:
#!cat /etc/apt/sources.list.d/cassandra.list

In [8]:
!apt update > null
!apt install cassandra > null

## Start the Service

In [9]:
#!service cassandra stop
# the ulimit error does not matter
!service cassandra restart

 * Restarting Cassandra cassandra
/etc/init.d/cassandra: 72: ulimit: error setting limit (Operation not permitted)
   ...done.


In [10]:
# keep checking service status until you see [+] next to cassandra
# then wait for at least 60 seconds MORE .... count 1 to 60 if you are feeling impatient
!service  --status-all 

 [ + ]  cassandra
 [ - ]  dbus
 [ ? ]  hwclock.sh
 [ ? ]  kmod
 [ - ]  ntp
 [ - ]  procps
 [ - ]  rsync
 [ - ]  udev
 [ - ]  x11-common


In [11]:
# this will give an error if the cassandra service has not started
# Wait for some time for the service to start
# If required reboot system or stop, restart cassandra service
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  127.0.0.1  69.05 KiB  16      100.0%            e31e5204-262e-4b06-a7c7-97450a3a1e32  rack1



In [ ]:
# basic test
#!cqlsh

# Basic Commands 
how to execute commands through scripts https://stackoverflow.com/questions/25286253/executing-cql-through-shell-script <br>
some examples https://techfossguru.com/apache-cassandra-python-step-step-guide-ubuntu-example/

In [ ]:
!cqlsh -e "help" 


Documented shell commands:
CAPTURE  CLS          COPY  DESCRIBE  EXPAND  LOGIN   SERIAL  SOURCE   UNICODE
CLEAR    CONSISTENCY  DESC  EXIT      HELP    PAGING  SHOW    TRACING

CQL help topics:
AGGREGATES               CREATE_KEYSPACE           DROP_TRIGGER      TEXT     
ALTER_KEYSPACE           CREATE_MATERIALIZED_VIEW  DROP_TYPE         TIME     
ALTER_MATERIALIZED_VIEW  CREATE_ROLE               DROP_USER         TIMESTAMP
ALTER_TABLE              CREATE_TABLE              FUNCTIONS         TRUNCATE 
ALTER_TYPE               CREATE_TRIGGER            GRANT             TYPES    
ALTER_USER               CREATE_TYPE               INSERT            UPDATE   
APPLY                    CREATE_USER               INSERT_JSON       USE      
ASCII                    DATE                      INT               UUID     
BATCH                    DELETE                    JSON            
BEGIN                    DROP_AGGREGATE            KEYWORDS        
BLOB                     DROP_COLUMNF

In [12]:
!cqlsh -e "CREATE KEYSPACE praxis WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };"


Warnings :
Your replication factor 3 for keyspace praxis is higher than the number of nodes 1



In [13]:
# since we are not specifying keyspace separately, we have to prefix table name with keyspace
!cqlsh -e "CREATE TABLE praxis.users (user_id int PRIMARY KEY, first_name text, last_name text );"

In [14]:
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (1, 'Prithwis', 'Mukerjee');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (2, 'Sanniti', 'Chatterjee');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (3, 'Amrita', 'Ghosh');"

In [15]:
!cqlsh -e "select * from praxis.users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


In [ ]:
#another way to specifiy keyspace
!cqlsh -e "use praxis;\
           select * from users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


In [ ]:
# easier way to specify keyspace, similar specifying database in mysql
!cqlsh -k praxis -e "select * from users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


# Python
https://docs.datastax.com/en/developer/python-driver/3.25/

In [26]:
!pip install cassandra-driver > null

In [29]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'],port=9042)
session = cluster.connect('praxis',wait_for_all_pools=True)
session.execute('USE praxis')

In [30]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Prithwis Mukerjee
2 Sanniti Chatterjee
3 Amrita Ghosh


In [ ]:
session.execute("INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (4, 'Charanpreet', 'Singh');")

In [ ]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Prithwis Mukerjee
2 Sanniti Chatterjee
4 Charanpreet Singh
3 Amrita Ghosh


In [ ]:
session.execute("update praxis.users set first_name = 'Dr Prithwis' where user_id = 1;")

In [ ]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Dr Prithwis Mukerjee
2 Sanniti Chatterjee
4 Charanpreet Singh
3 Amrita Ghosh


#Tutorial from FreeCodeCamp <br>
https://www.freecodecamp.org/news/the-apache-cassandra-beginner-tutorial/ <br>
alternatively from this [saved file](https://github.com/Praxis-QR/BDSN/blob/main/Documents/The%20Apache%20Cassandra%20Beginner%20Tutorial.pdf)

In [ ]:
#!cqlsh -e "CREATE KEYSPACE learn_cassandra WITH REPLICATION = { 'class' : 'NetworkTopologyStrategy','datacenter1' : 3  };"

In [ ]:
!cqlsh -k praxis -e "CREATE TABLE users_by_country \
(country text,\
 user_email text,\
 first_name text,\
 last_name text,\
 age smallint,\
 PRIMARY KEY ((country), user_email));"

In [ ]:
!cqlsh -k praxis -e "\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age) \
  VALUES('US', 'john@email.com', 'John','Wick',55); \
\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'peter@email.com', 'Peter','Clark',65);\
\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'bob@email.com', 'Bob','Sandler',23);\
\
INSERT INTO users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'alice@email.com', 'Alice','Brown',26);\
"

In [ ]:
!cqlsh -k praxis -e "SELECT * FROM users_by_country WHERE country='UK';"


 country | user_email      | age | first_name | last_name
---------+-----------------+-----+------------+-----------
      UK | alice@email.com |  26 |      Alice |     Brown
      UK |   bob@email.com |  23 |        Bob |   Sandler
      UK | peter@email.com |  65 |      Peter |     Clark

(3 rows)


In [ ]:
!cqlsh -k praxis -e "drop table users_by_email;"
!cqlsh -k praxis -e "CREATE TABLE users_by_email (\
  user_email text,\
  country text,\
  first_name text,\
  last_name text,\
  age smallint,\
  PRIMARY KEY (user_email));"
  

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'praxis.users_by_email' doesn't exist"


In [ ]:
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email, country,first_name,last_name,age)  VALUES('john@email.com', 'US', 'John','Wick',55);"
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email,country,first_name,last_name,age)  VALUES('peter@email.com', 'UK', 'Peter','Clark',65);"
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email,country,first_name,last_name,age)  VALUES('bob@email.com', 'UK', 'Bob','Sandler',23);"
!cqlsh -k praxis -e "INSERT INTO users_by_email (user_email,country,first_name,last_name,age)  VALUES('alice@email.com', 'UK', 'Alice','Brown',26);"

In [ ]:
!cqlsh -k praxis -e "SELECT * FROM users_by_email WHERE user_email='alice@email.com';"


 user_email      | age | country | first_name | last_name
-----------------+-----+---------+------------+-----------
 alice@email.com |  26 |      UK |      Alice |     Brown

(1 rows)


In [ ]:
!cqlsh -k praxis -e "SELECT * FROM users_by_email WHERE age=26 ALLOW FILTERING;"
!cqlsh -k praxis -e "SELECT * FROM users_by_email WHERE age=26;"


 user_email      | age | country | first_name | last_name
-----------------+-----+---------+------------+-----------
 alice@email.com |  26 |      UK |      Alice |     Brown

(1 rows)
<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


In [ ]:
!cqlsh -k praxis -e "CREATE TABLE todo_by_user_email (user_email text,name text,creation_date timestamp,PRIMARY KEY ((user_email), creation_date)) WITH CLUSTERING ORDER BY (creation_date DESC) AND compaction = { 'class' :  'LeveledCompactionStrategy'};"

In [ ]:
!cqlsh -k praxis -e "\
CREATE TABLE todos_shared_by_target_user_email ( \
    target_user_email text, \
    source_user_email text, \
    creation_date timestamp, \
    name text, \
    PRIMARY KEY ((target_user_email), creation_date) \
) WITH CLUSTERING ORDER BY (creation_date DESC) \
AND compaction = { 'class' :  'LeveledCompactionStrategy'  };"

In [ ]:
!cqlsh -k praxis -e "\
CREATE TABLE todos_shared_by_source_user_email ( \
    target_user_email text, \
    source_user_email text, \
    creation_date timestamp, \
    name text, \
    PRIMARY KEY ((source_user_email), creation_date) \
) WITH CLUSTERING ORDER BY (creation_date DESC) \
AND compaction = { 'class' :  'LeveledCompactionStrategy'  };"

In [ ]:
!cqlsh -k praxis -e "\
BEGIN BATCH \
  INSERT INTO todo_by_user_email (user_email,creation_date,name) VALUES('alice@email.com', toTimestamp(now()), 'My first todo entry') \
  INSERT INTO todos_shared_by_target_user_email (target_user_email, source_user_email,creation_date,name) VALUES('bob@email.com', 'alice@email.com',toTimestamp(now()), 'My first todo entry') \
  INSERT INTO todos_shared_by_source_user_email (target_user_email, source_user_email,creation_date,name) VALUES('alice@email.com', 'bob@email.com', toTimestamp(now()), 'My first todo entry') \
APPLY BATCH;"

In [ ]:
!cqlsh -k praxis -e "SELECT * FROM todos_shared_by_target_user_email WHERE target_user_email='bob@email.com';"


 target_user_email | creation_date                   | name                | source_user_email
-------------------+---------------------------------+---------------------+-------------------
     bob@email.com | 2021-12-03 04:13:45.357000+0000 | My first todo entry |   alice@email.com

(1 rows)


In [ ]:
!cqlsh -k praxis -e "SELECT * FROM todo_by_user_email WHERE user_email='alice@email.com';"


 user_email      | creation_date                   | name
-----------------+---------------------------------+---------------------
 alice@email.com | 2021-12-03 04:13:45.356000+0000 | My first todo entry

(1 rows)


#Cassandra Collections <br>
Set, List & Map in CQL with Example <br>
https://www.guru99.com/cassandra-collections-tutorial-set-list-map.html

## Set

In [ ]:
!cqlsh -k praxis -e "drop table teacher01;"
!cqlsh -k praxis -e "\
Create table Teacher01 \
( \
id int, \
Name text, \
Email set<text>, \
Primary key(id) \
);\
"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'praxis.teacher01' doesn't exist"


In [ ]:
!cqlsh -k praxis  -e "insert into Teacher01(id,Name,Email) \
 values(1,'prithwis',{{'abc@gmail.com','xyz@hotmail.com'}});"

In [ ]:
!cqlsh -k praxis  -e "insert into Teacher01(id,Name,Email) \
values(2,'charan',{{'abc1@gmail.com','xyz1@hotmail.com','xyz2@hotmail.com'}});"

In [ ]:
!cqlsh -k praxis -e " select * from Teacher01;"


 id | email                                                      | name
----+------------------------------------------------------------+----------
  1 |                       {'abc@gmail.com', 'xyz@hotmail.com'} | prithwis
  2 | {'abc1@gmail.com', 'xyz1@hotmail.com', 'xyz2@hotmail.com'} |   charan

(2 rows)


## List

In [ ]:
!cqlsh -k praxis -e "drop table teacher02;"
!cqlsh -k praxis -e "\
Create table Teacher02 \
( \
id int, \
Name text, \
Courses list<text>, \
Primary key(id) \
);\
"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Table 'praxis.teacher02' doesn't exist"


In [ ]:
!cqlsh -k praxis  -e "insert into Teacher02(id,Name,Courses) values(1,'prithwis',{['FDS','Python']});"
!cqlsh -k praxis  -e "insert into Teacher02(id,Name,Courses) values(2,'charan',{['Comms','English','Interview']});"

In [ ]:
!cqlsh -k praxis -e "select * from teacher02;"


 id | courses                           | name
----+-----------------------------------+----------
  1 |                 ['FDS', 'Python'] | prithwis
  2 | ['Comms', 'English', 'Interview'] |   charan

(2 rows)


## Maps

In [23]:
!cqlsh -k praxis -e "drop table if exists courses;"
!cqlsh -k praxis -e "\
Create table courses \
( \
id int, \
Name text, \
Teachers map<text,text>, \
Primary key(id) \
);\
"

In [24]:
!cqlsh -k praxis  -e "insert into courses(id,Name,Teachers) values(100,'PGDM',{{'Stats':'SRC','FDM':'Atanu'}});"
!cqlsh -k praxis  -e "insert into courses(id,Name,Teachers) values(200,'PGDS',{{'Stats':'SRC','ML':'SDG', 'BDSM':'PM'}});"
!cqlsh -k praxis -e "select * from courses;"


 id  | name | teachers
-----+------+---------------------------------------------
 200 | PGDS | {'BDSM': 'PM', 'ML': 'SDG', 'Stats': 'SRC'}
 100 | PGDM |            {'FDM': 'Atanu', 'Stats': 'SRC'}

(2 rows)


In [ ]:
#!cqlsh -k praxis -e "describe Teacher02;"

## Python

In [ ]:
rows = session.execute('SELECT * FROM teacher01')
for row in rows:
        print(row)
        emails = row.email
        print(emails)
        emails2 = list(emails)
        for im in emails2:
            print(im)

Row(id=1, email=SortedSet(['abc@gmail.com', 'xyz@hotmail.com']), name='prithwis')
SortedSet(['abc@gmail.com', 'xyz@hotmail.com'])
abc@gmail.com
xyz@hotmail.com
Row(id=2, email=SortedSet(['abc1@gmail.com', 'xyz1@hotmail.com', 'xyz2@hotmail.com']), name='charan')
SortedSet(['abc1@gmail.com', 'xyz1@hotmail.com', 'xyz2@hotmail.com'])
abc1@gmail.com
xyz1@hotmail.com
xyz2@hotmail.com


In [ ]:
rows = session.execute('SELECT * FROM teacher02')
for row in rows:
        print(row)
        courses = row.courses
        print(courses)
        for c in courses:
            print(c)

Row(id=1, courses=['FDS', 'Python'], name='prithwis')
['FDS', 'Python']
FDS
Python
Row(id=2, courses=['Comms', 'English', 'Interview'], name='charan')
['Comms', 'English', 'Interview']
Comms
English
Interview


In [ ]:
rows = session.execute('SELECT * FROM courses')
for row in rows:
        print(row)
        print(row.teachers) # this is a dictionary and can be used as such
        for k,v in row.teachers.items():
            print(k,v)
        
        

Row(id=200, name='PGDS', teachers=OrderedMapSerializedKey([('BDSM', 'PM'), ('ML', 'SDG'), ('Stats', 'SRC')]))
{'BDSM': 'PM', 'ML': 'SDG', 'Stats': 'SRC'}
BDSM PM
ML SDG
Stats SRC
Row(id=100, name='PGDM', teachers=OrderedMapSerializedKey([('FDM', 'Atanu'), ('Stats', 'SRC')]))
{'FDM': 'Atanu', 'Stats': 'SRC'}
FDM Atanu
Stats SRC


# Bulk Data Load

In [4]:
!gdown https://drive.google.com/uc?id=1T7teYg1_SXzZLbh8SOF0UIzpyK-L1aZL
!cat Employee.csv

Downloading...
From: https://drive.google.com/uc?id=1T7teYg1_SXzZLbh8SOF0UIzpyK-L1aZL
To: /content/Employee.csv
100% 734/734 [00:00<00:00, 1.28MB/s]


In [46]:
# 
!cqlsh -k praxis -e "drop table if exists emp;"
!cqlsh -k praxis -e "\
    CREATE TABLE emp (\
    empid int PRIMARY KEY, \
    lastname text,\
    firsname text,\
    jobdesc text,\
    joindate text,\
    salary int,\
    comm decimal,\
    deptid text \
    );\
"

In [47]:
!cqlsh -k praxis -e "select * from emp;"


 empid | comm | deptid | firsname | jobdesc | joindate | lastname | salary
-------+------+--------+----------+---------+----------+----------+--------


(0 rows)


In [48]:
!cqlsh -k praxis -e "\
    COPY emp (empid,lastname,firsname,jobdesc,joindate,salary,comm,deptid) \
    FROM 'Employee.csv' \
    WITH HEADER = TRUE;\
"

Using 1 child processes

Starting copy of praxis.emp with columns [empid, lastname, firsname, jobdesc, joindate, salary, comm, deptid].
Processed: 12 rows; Rate:      19 rows/s; Avg. rate:      29 rows/s
12 rows imported from 1 files in 0.421 seconds (0 skipped).


In [49]:
!cqlsh -k praxis -e "select * from emp;"


 empid  | comm | deptid | firsname | jobdesc    | joindate   | lastname    | salary
--------+------+--------+----------+------------+------------+-------------+--------
 546223 | 0.09 |     10 |  Narayan |  Secretary | 2005-12-04 | Karthikeyan |  40000
 223112 | 0.04 |     30 |    Sania |    Cus_Rep | 2001-11-19 |       Mirza |  25000
 989007 | 0.03 |     40 |   Sourav |    Cus_Rep | 2002-01-01 |     Ganguly |  20000
 865477 | 0.02 |     20 |  Madhuri |      Clerk | 2002-04-04 |     Dikshit |  10000
 123980 | 0.02 |     40 | Mahender |      Clerk | 2004-10-09 |       Dhoni |   9000
 897889 | 0.05 |     20 | Virender |    Cus_Rep | 2005-01-02 |      Sehwag |  15000
 349870 | 0.06 |     40 |     Rani |    Manager | 2005-05-04 |   Mukherjee |  25000
 239456 | 0.07 |     20 | Shahrukh |    Manager | 2004-01-03 |        Khan |  30000
 742866 |  0.1 |     10 |  Amitabh |  Executive | 2003-03-10 |     Bacchan |  50000
 997445 | 0.02 |     30 | Jagmohan |      Clerk | 2001-07-01 |      Dalmia

In [50]:
rows = session.execute("SELECT * from emp where jobdesc = 'Cus_Rep'")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)

InvalidRequest: ignored

In [51]:
#Create index IndexName on KeyspaceName.TableName(ColumnName);
!cqlsh -k praxis -e "create index IndexName on emp (jobdesc);"

In [52]:
rows = session.execute("SELECT * from emp where jobdesc = 'Cus_Rep'")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)
        #emails2 = list(emails)
        #for im in emails2:
        #    print(im)

Row(empid=223112, comm=Decimal('0.04'), deptid='30', firsname='Sania', jobdesc='Cus_Rep', joindate='2001-11-19', lastname='Mirza', salary=25000)
Mirza Sania
Row(empid=989007, comm=Decimal('0.03'), deptid='40', firsname='Sourav', jobdesc='Cus_Rep', joindate='2002-01-01', lastname='Ganguly', salary=20000)
Ganguly Sourav
Row(empid=897889, comm=Decimal('0.05'), deptid='20', firsname='Virender', jobdesc='Cus_Rep', joindate='2005-01-02', lastname='Sehwag', salary=15000)
Sehwag Virender


In [55]:
rows = session.execute("SELECT * from emp where jobdesc like '%Manager%'")
for row in rows:
        print(row)
        LastName = row.lastname
        FirstName = row.firsname
        print(LastName, FirstName)
        #emails2 = list(emails)
        #for im in emails2:
        #    print(im)

InvalidRequest: ignored

In [56]:
!cqlsh -k praxis -e "\
CREATE CUSTOM INDEX  fn_prefix ON emp (jobdesc) USING 'org.apache.cassandra.index.sasi.SASIIndex';\
"

<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="SASI indexes are disabled. Enable in cassandra.yaml to use."


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)